<a href="https://colab.research.google.com/github/shriram-stack/Cancer_Detection/blob/main/IRIS_damage_prediction_using_CNN_and_Image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import os

# Check TensorFlow version
print("TensorFlow Version:", tf.__version__)

# Mount Google Drive (optional, if your dataset is stored there)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Simulate a small dataset (replace this with your actual iris image dataset)
def generate_synthetic_iris_images(num_images, img_size=(128, 128)):
    images = []
    labels = []  # 0: no damage, 1: damaged
    for _ in range(num_images):
        # Randomly generate an "iris-like" image
        img = np.random.rand(img_size[0], img_size[1], 3) * 255
        img = img.astype(np.uint8)
        label = np.random.randint(0, 2)  # Binary classification
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)

# Generate synthetic data
num_images = 100  # Increase this with real data
X, y = generate_synthetic_iris_images(num_images)

# Preprocessing function
def preprocess_image(img):
    # Noise reduction (Gaussian blur)
    img = cv2.GaussianBlur(img, (5, 5), 0)
    # Contrast enhancement (CLAHE)
    if len(img.shape) == 3:
        img_lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        img_lab[:,:,0] = clahe.apply(img_lab[:,:,0])
        img = cv2.cvtColor(img_lab, cv2.COLOR_LAB2RGB)
    # Normalization
    img = img / 255.0
    return img

# Apply preprocessing to all images
X_processed = np.array([preprocess_image(img) for img in X])

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.2, random_state=42)

print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)

In [ ]:
# Define the CNN model
def create_cnn_model(input_shape=(128, 128, 3)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    return model

# Create and compile the model
model = create_cnn_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

In [ ]:
# Data augmentation (optional, to improve generalization)
datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.15, horizontal_flip=True)
datagen.fit(X_train)

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=10,  # Increase epochs for real data
                    validation_data=(X_val, y_val))

# Plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Evaluate CNN model
y_pred_cnn = (model.predict(X_val) > 0.5).astype(int)
print("CNN Classification Report:")
print(classification_report(y_val, y_pred_cnn))

# SVM comparison
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Flatten images for SVM
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_val.shape[0], -1)

svm = SVC(kernel='linear')
svm.fit(X_train_flat, y_train)
y_pred_svm = svm.predict(X_val_flat)
print("SVM Classification Report:")
print(classification_report(y_val, y_pred_svm))

# Random Forest (optional, uncomment to use)
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(n_estimators=100)
# rf.fit(X_train_flat, y_train)
# y_pred_rf = rf.predict(X_val_flat)
# print("Random Forest Classification Report:")
# print(classification_report(y_val, y_pred_rf))

In [ ]:
# Save the CNN model
model.save('/content/drive/MyDrive/iris_damage_cnn_model.h5')
print("Model saved successfully!")